# Media Optimization

The model is imported

In [99]:
import pandas as pd
import numpy as np

In [100]:
%run 01_loading_model.ipynb

### The components of the standard medium and their relative abundance 

In [101]:
model.medium

{'r_1654': 1000.0,
 'r_1714': 1.0,
 'r_1832': 1000.0,
 'r_1861': 1000.0,
 'r_1992': 1000.0,
 'r_2005': 1000.0,
 'r_2020': 1000.0,
 'r_2049': 1000.0,
 'r_2060': 1000.0,
 'r_2100': 1000.0,
 'r_4593': 1000.0,
 'r_4594': 1000.0,
 'r_4595': 1000.0,
 'r_4596': 1000.0,
 'r_4597': 1000.0,
 'r_4600': 1000.0}

To be able to see which metabolites are responsible for the exchange reactions their name is extracted with the following for loop:

In [102]:
for x, y in model.medium.items():
    print(model.reactions.get_by_id(x).id,model.reactions.get_by_id(x).name,"\t",y)

r_1654 ammonium exchange 	 1000.0
r_1714 D-glucose exchange 	 1.0
r_1832 H+ exchange 	 1000.0
r_1861 iron(2+) exchange 	 1000.0
r_1992 oxygen exchange 	 1000.0
r_2005 phosphate exchange 	 1000.0
r_2020 potassium exchange 	 1000.0
r_2049 sodium exchange 	 1000.0
r_2060 sulphate exchange 	 1000.0
r_2100 water exchange 	 1000.0
r_4593 chloride exchange 	 1000.0
r_4594 Cu2(+) exchange 	 1000.0
r_4595 Mn(2+) exchange 	 1000.0
r_4596 Zn(2+) exchange 	 1000.0
r_4597 Mg(2+) exchange 	 1000.0
r_4600 Ca(2+) exchange 	 1000.0


To make it more understandable, the import reactions and the name of the metabolites are extracted. 
Fist we identify the reaction annotated as growth/biomass, which is r_2111. This corresponds to:

In [103]:
model.reactions.r_2111
model.optimize()

,fluxes,reduced_costs
r_0001,0.000000,-1.667286e-02
r_0002,0.000000,-1.667286e-02
r_0003,0.000000,-0.000000e+00
r_0004,0.000000,-1.734723e-18
r_0005,0.062686,0.000000e+00
...,...,...
CrtI,0.000000,0.000000e+00
DM_dmapp,0.000000,0.000000e+00
DM_ggpp,0.000000,0.000000e+00
DM_phyto,0.000000,0.000000e+00


In [104]:
with model:
    solution = model.optimize()
    print('Maximum theoretical biomass productivity:',solution.fluxes["r_2111"], "\h") 

    model.objective = model.reactions.CrtI
    lyco_production = model.optimize().objective_value
    print('Maximum theoretical productivity of lycopene:', lyco_production, 'mmol/gDW*h')
    print("Theoretical max. yield of lycopene:", lyco_production / (-1*model.reactions.r_1714.flux), "mmol-lyco / mmol-glc")

Maximum theoretical biomass productivity: 0.08374778664999817 \h
Maximum theoretical productivity of lycopene reaction: 0.16915906286462645 mmol/gDW*h
Theoretical max. yield of lycopene: 0.16915906286462645 mmol-lyco / mmol-glc


### Changing media

In order to analyse the effect of the medium on the productivities, different modifications to the medium are attempted. 
Initially, the medium is updated to a more ideal medium for S. cerevisiae: the YEPD medim

#### YEPD-mimicked media

We have to identify the ID of all amino acid exchange reactions. This is done with the following for-loop

In [105]:
 aa_list=["L-alanine", "L-arginine", "L-asparagine", "L-aspartate", "L-cysteine", "L-glucitol", "L-glutamate", "L-glutamine", "L-glycine", "L-histidine", "L-homoserine", "L-isoleucine", "L-leucine", "L-lysine", "L-methionine", "L-phenylalanine", "L-proline", "L-serine", "L-sorbose", "L-threonine", "L-tryptophan", "L-tyrosine", "L-valine"]
for aa in aa_list:
    for m in model.reactions.query(r'^'+aa+" exchange", 'name'):
        print(m.id,m.name)

r_1873 L-alanine exchange
r_1879 L-arginine exchange
r_1880 L-asparagine exchange
r_1881 L-aspartate exchange
r_1883 L-cysteine exchange
r_1886 L-glucitol exchange
r_1889 L-glutamate exchange
r_1891 L-glutamine exchange
r_1810 L-glycine exchange
r_1893 L-histidine exchange
r_1896 L-homoserine exchange
r_1897 L-isoleucine exchange
r_1899 L-leucine exchange
r_1900 L-lysine exchange
r_1902 L-methionine exchange
r_1903 L-phenylalanine exchange
r_1904 L-proline exchange
r_1906 L-serine exchange
r_1909 L-sorbose exchange
r_1911 L-threonine exchange
r_1912 L-tryptophan exchange
r_1913 L-tyrosine exchange
r_1914 L-valine exchange


With the identified IDs, the medium content is updated

In [106]:
YEPDmedium = model.medium
YEPDmedium['r_1714'] = 20
YEPDmedium['r_1873'] = 10
YEPDmedium['r_1879'] = 10
YEPDmedium['r_1880'] = 10
YEPDmedium['r_1881'] = 10
YEPDmedium['r_1883'] = 10
YEPDmedium['r_1889'] = 10
YEPDmedium['r_1891'] = 10
YEPDmedium['r_1810'] = 10
YEPDmedium['r_1893'] = 10
YEPDmedium['r_1897'] = 10
YEPDmedium['r_1899'] = 10
YEPDmedium['r_1900'] = 10
YEPDmedium['r_1902'] = 10
YEPDmedium['r_1903'] = 10
YEPDmedium['r_1903'] = 10
YEPDmedium['r_1904'] = 10
YEPDmedium['r_1906'] = 10
YEPDmedium['r_1911'] = 10
YEPDmedium['r_1912'] = 10
YEPDmedium['r_1913'] = 10
YEPDmedium['r_1914'] = 10


#### Poductivites on YEPD medium

To analyse the productivities on this medium, the performance is tested

In [107]:
model.medium = YEPDmedium
with model:
    solution = model.optimize()
    print('Maximum theoretical biomass productivity:',solution.fluxes["r_2111"], "\h") 

    model.objective = model.reactions.CrtI
    lyco_production = model.optimize().objective_value
    print('Theoretical max. productivity of lycopene:', lyco_production, 'mmol/gDW*h')
    print("Theoretical max. yield of lycopene:", lyco_production / (-1*model.reactions.r_1714.flux), "mmol-lyco / mmol-glc")

Maximum theoretical biomass productivity: 10.237513608012602 \h
Theoretical max. productivity of lycopene: 16.709647633330416 mmol/gDW*h
Theoretical max. yield of lycopene: 0.8354823816665208 mmol-lyco / mmol-glc


Very clearly it can be seen that by adding amino acids to the medium, lycopene productivity increases excessively. This is likely due to biosynthesis of amino acids requires a lot of the metabolites generated throughout central metabolism. When the requirement for endogenous synthesis decreases, the flux can be directed towards increased production of other metabolites, such as lycopene

#### Changing the carbon source in the medium

To further improve the medium, the carbon source is furhter examined. The carbon source can be changed in the medium by chinging the flux bounds of the apropriate exchange reactions. The following code is an example of how the exchange reactions are identified.

In [108]:
for m in model.reactions.query("succinate exchange", 'name'):
        print(m.id,m.name)

r_2056 succinate exchange


Initially, the glucose is increased from 1 to 10: 

In [109]:
medium = model.medium
with model:
    medium['r_1714'] = 10 #Glucose exchange 
    model.medium = medium
    solution1= model.optimize()
    print('Maximum theoretical biomass productivity:',solution1.fluxes["r_2111"], "\h") 
    
    model.objective = model.reactions.CrtI
    lyco_production = model.optimize().objective_value
    print('Maximum theoretical productivity of lycopene reaction:', lyco_production, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 9.371403141493689 \h
Maximum theoretical productivity of lycopene reaction: 14.9402786212709 mmol/gDW*h


Then different carbon sources are added to the medium while glucose is removed

In [110]:
medium = model.medium
with model:
    medium['r_1714'] = 0 #Glucose exchange 
    medium['r_2056'] = 10 #Succinate exchange
    model.medium = medium
    solution2= model.optimize()
    print('Maximum theoretical biomass productivity:',solution2.fluxes["r_2111"], "\h") 
    
    model.objective = model.reactions.CrtI
    lyco_production = model.optimize().objective_value
    print('Maximum theoretical productivity of lycopene reaction:', lyco_production, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 8.760445060484907 \h
Maximum theoretical productivity of lycopene reaction: 14.160636426922654 mmol/gDW*h


Changing the carbon source to succinate led to a decrease in growth rate.

In [111]:
medium = model.medium
with model:
    medium['r_1714'] = 0 #Glucose exchange 
    medium['r_1709'] = 10 #Fructose exchange
    model.medium = medium
    solution3= model.optimize()
    print('Maximum theoretical biomass productivity:',solution3.fluxes["r_2111"], "\h") 
    
    model.objective = model.reactions.CrtI
    lyco_production = model.optimize().objective_value
    print('Maximum theoretical productivity of lycopene reaction:', lyco_production, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 9.360908802688375 \h
Maximum theoretical productivity of lycopene reaction: 14.916649108238401 mmol/gDW*h


Changing the carbon source to fructose gives a growth rate similar to glucose

In [112]:
medium = model.medium
with model:
    medium['r_1714'] = 0 #Glucose exchange 
    medium['r_2033'] = 10 #Pyruvate exchange
    model.medium = medium
    solution4= model.optimize()
    print('Maximum theoretical biomass productivity:',solution4.fluxes["r_2111"], "\h") 
    
    model.objective = model.reactions.CrtI
    lyco_production = model.optimize().objective_value
    print('Maximum theoretical productivity of lycopene reaction:', lyco_production, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 8.575127321611605 \h
Maximum theoretical productivity of lycopene reaction: 13.886697969642164 mmol/gDW*h


Pyruvate however does not seem to increase the productivity

In [113]:
medium = model.medium
with model:
    medium['r_1714'] = 0 #Glucose exchange 
    medium['r_1687'] = 10 #Citrate exchange
    model.medium = medium
    solution5= model.optimize()
    print('Maximum theoretical biomass productivity:',solution5.fluxes["r_2111"], "\h") 
    
    model.objective = model.reactions.CrtI
    lyco_production = model.optimize().objective_value
    print('Maximum theoretical productivity of lycopene reaction:', lyco_production, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 8.986853650905225 \h
Maximum theoretical productivity of lycopene reaction: 14.513321377066154 mmol/gDW*h


## Text for report: 
In the model, the default medium is based on the most essential components for growth. Glucose is the carbon source, ammonium is the nitrogen source, and the most essential ions and trace metals are also included. In this medium, biosynthesis of all amino acids and various other central metabolites are required. Thus, a lot of the carbon is used for biosynthesis of central metabolites instead of lycopene synthesis. To optimize the medium, we have changed the medium, so it mimics the YEPD medium, which often is applied for fungal growth. This was done by adding all amino acids to the medium, in order to limit the requirement for biosynthesis. In addition, the glucose uptake is increased to 20 mmol/gDW/h to improve the growth of S. cerevisiae. 
A change of the medium significantly increases the biomass productivity and the productivity of lycopene. Biomass productivity is increased 119-fold and lycopene productivity is increased by 99-fold. The maximum theoretical yield of lycopene is increased by a factor 5. 
In addition to examining the impact of the amino acids on the growth of S. cerevisiae, the carbon source was also changed. The carbon sources that were tested are glucose, fructose, succinate, pyruvate and citrate. Glucose and fructose resulted in the highest biomass and lycopene productivity, but no major differences between the various carbon sources were observed. 
